In [55]:
import requests
import time
import json

# URLs for local and remote testing
LOCAL_URL = "http://127.0.0.1:8001"
BASE_URL = "http://51.159.160.67:8000"

# Choose the URL to use for testing
USE_LOCAL = True
URL = LOCAL_URL if USE_LOCAL else BASE_URL

# Paths to test files
test_sdf = "/root/projects/DiffDock/examples/1a46_ligand.sdf"
test_pdb = "/root/projects/DiffDock/examples/1a46_protein_processed.pdb"
test_zip = "/root/projects/DiffDock/data/1a0q_test_data.zip"

# Test the /ping endpoint
def test_ping():
    response = requests.get(f"{URL}/ping")
    print("Ping response:", response.json())

# Test the /inference endpoint with a sample PDB and SDF file
def test_inference():
    with open(test_pdb, "rb") as pdb_file, open(test_sdf, "rb") as sdf_file:
        files = {
            "pdb_file": pdb_file,
            "sdf_file": sdf_file
        }
        body = {
            "input": {
                "protein_path": "1a46_protein.pdb",
                "ligand_description": "1a46_ligand.sdf"
            },
            "config": {
                "inference_steps": 20,
                "samples_per_complex": 10
            }
        }
        response = requests.post(f"{URL}/inference", files=files, data={"body": json.dumps(body)})
        print("Inference response:", response.json())
        return response.json().get("task_id")

# Test the /inference/status endpoint
def test_inference_status(task_id):
    response = requests.get(f"{URL}/inference/status/{task_id}")
    print("Inference status response:", response.json())
    return response.json()

# Test the /inference/download endpoint
def test_inference_download(task_id):
    response = requests.get(f"{URL}/inference/download/{task_id}")
    with open(f"{task_id}_output.zip", "wb") as f:
        f.write(response.content)
    print(f"Downloaded result for task {task_id} to {task_id}_output.zip")

# Test the /inference/zip/ endpoint with a sample zip file
def test_inference_zip():
    with open(test_zip, "rb") as zip_file:
        files = {"zip_file": zip_file}
        config = {
            "inference_steps": 20,
            "samples_per_complex": 10
        }
        response = requests.post(f"{URL}/inference/zip/", files=files, data={"config": json.dumps(config)})
        print("Inference zip response:", response.json())
        return response.json().get("task_id")

# Run tests
test_ping()
task_id = test_inference()

while True:
    status = test_inference_status(task_id)
    if status.get('status') == 'completed':
        test_inference_download(task_id)
        break
    elif status.get('status') == 'failed':
        print(f"Task failed: {status.get('error')}")
        break
    time.sleep(10)

task_id_zip = test_inference_zip()

while True:
    status = test_inference_status(task_id_zip)
    if status.get('status') == 'completed':
        test_inference_download(task_id_zip)
        break
    elif status.get('status') == 'failed':
        print(f"Task failed: {status.get('error')}")
        break
    time.sleep(10)


Ping response: {'message': 'pong'}
Inference response: {'message': 'Inference process started successfully', 'task_id': '888a1976-2a33-40c2-88fc-ecef27477b83', 'args': {'actual_steps': 19, 'ckpt': 'best_ema_inference_epoch_model.pt', 'confidence_ckpt': 'best_model_epoch75.pt', 'confidence_model_dir': './workdir/v1.1/confidence_model', 'different_schedules': False, 'inf_sched_alpha': 1, 'inf_sched_beta': 1, 'inference_steps': 20, 'initial_noise_std_proportion': 1.4601642460337794, 'limit_failures': 5, 'model_dir': './workdir/v1.1/score_model', 'no_final_step_noise': True, 'no_model': False, 'no_random': False, 'no_random_pocket': False, 'ode': False, 'old_confidence_model': True, 'old_score_model': False, 'resample_rdkit': False, 'samples_per_complex': 10, 'sigma_schedule': 'expbeta', 'temp_psi_rot': 0.9022615585677628, 'temp_psi_tor': 0.5946212391366862, 'temp_psi_tr': 0.727287304570729, 'temp_sampling_rot': 2.06391612594481, 'temp_sampling_tor': 7.044261621607846, 'temp_sampling_tr': 

In [32]:
# Jupyter Notebook Cell
!pip install requests

import requests

# Define the base URL of your API
base_url = 'http://51.159.189.40:8000'

# Function to test the root endpoint
def test_root_endpoint():
    url = base_url
    response = requests.get(url)
    if response.status_code == 200:
        print("Root endpoint is working.")
    else:
        print(f"Root endpoint failed with status code: {response.status_code}")
    print(response.json())

# Function to test a specific endpoint
def test_specific_endpoint(endpoint, payload=None):
    url = f"{base_url}{endpoint}"
    if payload:
        response = requests.post(url, json=payload)
    else:
        response = requests.get(url)
    if response.status_code == 200:
        print(f"Endpoint '{endpoint}' is working.")
    else:
        print(f"Endpoint '{endpoint}' failed with status code: {response.status_code}")
    print(response.json())

# Test the root endpoint
test_root_endpoint()

# Example of testing a specific endpoint
# Replace 'your-endpoint' with the actual endpoint and adjust the payload if necessary
test_specific_endpoint('/inference', payload={'key': 'value'})


Root endpoint failed with status code: 404
{'detail': 'Not Found'}
Endpoint '/inference' failed with status code: 422
{'detail': [{'type': 'missing', 'loc': ['body', 'input'], 'msg': 'Field required', 'input': {'key': 'value'}}, {'type': 'missing', 'loc': ['body', 'config'], 'msg': 'Field required', 'input': {'key': 'value'}}]}


## Test script for direct input

In [20]:
# test_script.py

import requests
from schemas import InferenceInput, InferenceConfig
import time

# Define the URL of the FastAPI server
api_url = "http://127.0.0.1:8000/inference/"
progress_url = "http://127.0.0.1:8000/inference/progress/"

# Example input data
inference_input = InferenceInput(
    protein_path="examples/6ahs_protein_processed.pdb",
    ligand_description="examples/6ahs_ligand.sdf"
)

inference_config = InferenceConfig(
    actual_steps=19,
    ckpt="best_ema_inference_epoch_model.pt",
    confidence_ckpt="best_model_epoch75.pt",
    confidence_model_dir="./workdir/v1.1/confidence_model",
    different_schedules=False,
    inf_sched_alpha=1,
    inf_sched_beta=1,
    inference_steps=20,
    initial_noise_std_proportion=1.4601642460337794,
    limit_failures=5,
    model_dir="./workdir/v1.1/score_model",
    no_final_step_noise=True,
    no_model=False,
    no_random=False,
    no_random_pocket=False,
    ode=False,
    old_filtering_model=True,
    old_score_model=False,
    resample_rdkit=False,
    samples_per_complex=10,
    sigma_schedule="expbeta",
    temp_psi_rot=0.9022615585677628,
    temp_psi_tor=0.5946212391366862,
    temp_psi_tr=0.727287304570729,
    temp_sampling_rot=2.06391612594481,
    temp_sampling_tor=7.044261621607846,
    temp_sampling_tr=1.170050527854316,
    temp_sigma_data_rot=0.7464326999906034,
    temp_sigma_data_tor=0.6943254174849822,
    temp_sigma_data_tr=0.9299802531572672,
    loglevel="WARNING",
    choose_residue=False,
    out_dir="results/user_inference",
    save_visualisation=False,
    batch_size=10
)

def test_inference(api_url, progress_url, inference_input, inference_config):
    response = requests.post(api_url, json={"input": inference_input.dict(), "config": inference_config.dict()})
    if response.status_code == 200:
        print("Inference initiated successfully")
        response_data = response.json()
        task_id = response_data["task_id"]
        print("Task ID:", task_id)
        
        # Check progress periodically
        while True:
            progress_response = requests.get(f"{progress_url}{task_id}")
            if progress_response.status_code == 200:
                progress_data = progress_response.json()
                print("Progress:", progress_data["progress"])
                if "Completed" in progress_data["progress"]:
                    break
            else:
                print("Failed to get progress")
            time.sleep(5)
            
    else:
        print("Failed to start inference")
        print("Status Code:", response.status_code)
        print("Response:", response.text)

if __name__ == "__main__":
    test_inference(api_url, progress_url, inference_input, inference_config)


/tmp/ipykernel_14389/3150982447.py:56: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  response = requests.post(api_url, json={"input": inference_input.dict(), "config": inference_config.dict()})


Inference initiated successfully
Task ID: d400ec06-e79e-45cd-a556-080693d5b039
Progress: No such task
Progress: No such task
Progress: No such task
Progress: No such task
Progress: No such task
Progress: No such task
Progress: No such task
Progress: No such task
Progress: No such task
Progress: No such task
Progress: No such task
Progress: No such task
Progress: No such task


KeyboardInterrupt: 

## Test script for zip input

In [1]:
# test_script_zip.py
import requests
import time
from pathlib import Path
from schemas import InferenceConfig
import json

# Define the URL of the FastAPI server
api_url = "http://127.0.0.1:8000/inference/zip/"
progress_url = "http://127.0.0.1:8000/inference/progress/"
download_url = "http://127.0.0.1:8000/inference/download/"

# Path to the test zip archive
zip_file_path = "/root/projects/DiffDock/data/1a0q_test_data.zip"

# Example inference configuration
inference_config = InferenceConfig(
    actual_steps=19,
    ckpt="best_ema_inference_epoch_model.pt",
    confidence_ckpt="best_model_epoch75.pt",
    confidence_model_dir="./workdir/v1.1/confidence_model",
    different_schedules=False,
    inf_sched_alpha=1,
    inf_sched_beta=1,
    inference_steps=20,
    initial_noise_std_proportion=1.4601642460337794,
    limit_failures=5,
    model_dir="./workdir/v1.1/score_model",
    no_final_step_noise=True,
    no_model=False,
    no_random=False,
    no_random_pocket=False,
    ode=False,
    old_confidence_model=True,
    old_score_model=False,
    resample_rdkit=False,
    samples_per_complex=10,
    sigma_schedule="expbeta",
    temp_psi_rot=0.9022615585677628,
    temp_psi_tor=0.5946212391366862,
    temp_psi_tr=0.727287304570729,
    temp_sampling_rot=2.06391612594481,
    temp_sampling_tor=7.044261621607846,
    temp_sampling_tr=1.170050527854316,
    temp_sigma_data_rot=0.7464326999906034,
    temp_sigma_data_tor=0.6943254174849822,
    temp_sigma_data_tr=0.9299802531572672,
    loglevel="WARNING",
    choose_residue=False,
    out_dir="results/user_inference",
    save_visualisation=False,
    batch_size=10
)

def test_inference_from_zip(api_url, progress_url, download_url, zip_file_path, config):
    with open(zip_file_path, "rb") as f:
        files = {"zip_file": f}
        config_json = json.dumps(config.model_dump())
        data = {"config": config_json}
        response = requests.post(api_url, files=files, data=data)
    
    if response.status_code == 200:
        print("Inference initiated successfully")
        response_data = response.json()
        task_id = response_data["task_id"]
        print("Task ID:", task_id)
        
        # Check progress periodically
        for _ in range(20):  # Wait long enough to ensure process has time to start
            progress_response = requests.get(f"{progress_url}{task_id}")
            if progress_response.status_code == 200:
                progress_data = progress_response.json()
                print("Progress:", progress_data["progress"])
                if progress_data["progress"] == "Completed":
                    break
            else:
                print("Failed to get progress")
            time.sleep(5)
        
        # Download the results
        download_response = requests.get(f"{download_url}{task_id}")
        if download_response.status_code == 200:
            output_zip_path = Path(f"{task_id}_output.zip")
            with open(output_zip_path, "wb") as f:
                f.write(download_response.content)
            print(f"Results downloaded successfully and saved to {output_zip_path}")
        else:
            print("Failed to download results")
            print("Status Code:", download_response.status_code)
            print("Response:", download_response.text)
    else:
        print("Failed to start inference")
        print("Status Code:", response.status_code)
        print("Response:", response.text)

# Run the test
if __name__ == "__main__":
    test_inference_from_zip(api_url, progress_url, download_url, zip_file_path, inference_config)


/root/miniconda3/envs/diffdock/lib/python3.9/site-packages/pydantic/_internal/_fields.py:149: UserWarning: Field "model_dir" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


Inference initiated successfully
Task ID: 102bda99-afe3-4c2e-9465-083a5105db82
Progress: No such task
Progress: No such task
Progress: No such task
Progress: No such task
Progress: No such task
Progress: No such task
Progress: No such task
Progress: No such task
Progress: No such task
Progress: No such task
Progress: No such task
Progress: No such task
Progress: No such task
Progress: No such task
Progress: No such task
Progress: No such task
Progress: No such task
Progress: No such task
Progress: No such task
Progress: No such task
Results downloaded successfully and saved to 102bda99-afe3-4c2e-9465-083a5105db82_output.zip
